In [1]:
import sys

sys.path.append("../")

import pandas as pd
import numpy as np
import datetime
import os
from pprint import pprint
import matplotlib.pyplot as plt
import time
import vectorbt as vbt
from time import time
import helpers as pth

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 20)

plt.style.use("classic")
# plt.rcParams["axes.grid"] = True
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.formatter.useoffset"] = False
plt.rcParams["axes.formatter.limits"] = [-1000000000, 1000000000]

/Users/e1211913/miniconda3/envs/trading/lib/python3.12/site-packages/telegram/utils/request.py:54: UserWarning: python-telegram-bot wasn't properly installed. Please refer to README.rst on how to properly install.
  warnings.warn(


In [2]:
I = pth.load_instruments()
I.keys()

dict_keys(['6B', 'MES'])

### Simple

In [50]:
fast = pth.to_indicator(I["6B"].avg)
slow = pth.to_indicator(I["6B"].avg_1)
price = pth.to_indicator(I["6B"].close)

entries = price.values_crossed_below(slow.values)
exits = price.values_crossed_above(slow.values)
exits.shape, entries.shape, I["6B"].shape

((58438,), (58438,), (58438, 15))

In [51]:
res = vbt.Portfolio.from_signals(I["6B"].close, entries=entries, exits=exits)
res.plot_value()

FigureWidget({
    'data': [{'hoverinfo': 'skip',
              'line': {'color': 'rgba(0, 0, 0, 0)', 'width': 0},
              'opacity': 0,
              'showlegend': False,
              'type': 'scatter',
              'uid': '72cc5586-040f-44dc-be61-85dc9d11d79f',
              'x': array([    0,     1,     2, ..., 58435, 58436, 58437]),
              'y': array([100., 100., 100., ..., 100., 100., 100.])},
             {'connectgaps': False,
              'fill': 'tonexty',
              'fillcolor': 'rgba(0, 128, 0, 0.3)',
              'hoverinfo': 'skip',
              'line': {'color': 'rgba(0, 0, 0, 0)', 'width': 0},
              'opacity': 0,
              'showlegend': False,
              'type': 'scatter',
              'uid': '61998d9e-f0fe-41b4-9539-3a2a416a64cd',
              'x': array([    0,     1,     2, ..., 58435, 58436, 58437]),
              'y': array([100.        , 100.        , 100.        , ..., 104.04633881,
                          104.04633881, 104.

### Multi Param / Multi Instrument

In [3]:
universe = pth.make_backtest_universe(I)
universe.head()

ticker,6B,MES
timestamp,,
2024-5-19 23:00:00.000000,1.2706,5527.25
2024-5-19 23:01:00.000000,1.2707,5526.50
2024-5-19 23:02:00.000000,1.2707,5527.00
2024-5-19 23:03:00.000000,1.2708,5528.25
2024-5-19 23:04:00.000000,1.2708,5528.50


In [39]:
import itertools

fast, slow = vbt.MA.run_combs(universe, np.arange(10, 240, 10))
entries = fast.ma_crossed_above(slow)
exits = fast.ma_crossed_below(slow)
entries.shape

(292443, 506)

In [40]:
res = vbt.Portfolio.from_signals(universe, entries, exits, freq="1D")
res.total_return().vbt.heatmap(
    x_level="ma_1_window", y_level="ma_2_window", slider_level="ticker", symmetric=True
)

/Users/e1211913/miniconda3/envs/trading/lib/python3.12/site-packages/jupyter_client/session.py:721: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant

/Users/e1211913/miniconda3/envs/trading/lib/python3.12/site-packages/jupyter_client/session.py:721: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant



FigureWidget({
    'data': [{'colorscale': [[0.0, '#0d0887'], [0.1111111111111111, '#46039f'],
                             [0.2222222222222222, '#7201a8'], [0.3333333333333333,
                             '#9c179e'], [0.4444444444444444, '#bd3786'],
                             [0.5555555555555556, '#d8576b'], [0.6666666666666666,
                             '#ed7953'], [0.7777777777777778, '#fb9f3a'],
                             [0.8888888888888888, '#fdca26'], [1.0, '#f0f921']],
              'hoverongaps': False,
              'hovertemplate': 'ma_1_window: %{x}<br>ma_2_window: %{y}<br>value: %{z}<extra></extra>',
              'name': '6B',
              'type': 'heatmap',
              'uid': '0b25bdbf-fab6-4d18-8c67-12e3a9c4370a',
              'visible': True,
              'x': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120, 130, 140,
                          150, 160, 170, 180, 190, 200, 210, 220, 230]),
              'y': array([ 10,  20,  30,  40,  50